In [2]:
import numpy as np
from numpy.linalg import eig
import scipy.linalg as la
import matplotlib.pyplot as plt 


In [3]:
class LPCSolver:
    def __init__(self) -> None:
        self.lpcPoints = []
        self.maxPoints = 200
        self.corvergenceRate = 10e-3
        self.correctionFactor = 1
        self.isDirectionForwards = True
        self.h = 80     #constant bandwidth parameter
        self.stepSize = 40
    
    def setMaxPoints(self, _maxPoints):
        self.maxPoints = _maxPoints

    def setBandwidth(self, _h):
        self.h = _h
    
    def setStepSize(self, _stepSize):
        self.stepSize= _stepSize
    
    def setPoints(self, _points, _amps):
        self.pointsXYZ = _points
        self.pointsAmpl = _amps

    # def setMaxAmplPoint(self, _points, _amps):
    #     if 
    #     self.maxAmplPoint = 

    def __computeWeight(self, location, pointAmpl, point) -> float:
        w = pointAmpl / (np.sqrt((2 * np.pi) ** 3) * (self.h ** 3))
        w *= np.exp(-1 /( 2* self.h * self.h ) * np.dot(point - location, point - location))
        return w 
    
    def __meanShift(self, location):#è la LOCAL MEAN
        vecSum = np.zeros(3)
        weightSum = 0
        for i,p in enumerate(self.pointsXYZ):
            w = self.__computeWeight(location, self.pointsAmpl[i], p)
            vecSum += w * p
            weightSum += w 
        return vecSum/weightSum

    def __lpcShift(self, location, prevEigVec):
        vecSum = np.zeros(shape=(3,3))
        weightSum = 0
        for i,p in enumerate(self.pointsXYZ):
            w = self.__computeWeight(location, self.pointsAmpl[i], p)
            vecSum += w * np.outer(p - location, p - location)#prodotto esterno
            weightSum += w
        covMatrix = vecSum / weightSum
        eigVal, eigVec = np.linalg.eig(covMatrix)
        lpcShiftEigVec = eigVec[np.argmax(eigVal)] / la.norm(eigVec[np.argmax(eigVal)])
        anglePenalization = 1
        if prevEigVec is not None:
            cosPhi = np.dot(lpcShiftEigVec, prevEigVec) / (la.norm(prevEigVec) * la.norm(lpcShiftEigVec))
            anglePenalization = np.abs(cosPhi) ** 2
            if (cosPhi < 0):
                lpcShiftEigVec = - lpcShiftEigVec
        if self.isDirectionForwards:
            lpcShiftPoint = location + lpcShiftEigVec * self.stepSize * anglePenalization
        else :
            lpcShiftPoint = location - lpcShiftEigVec * self.stepSize * anglePenalization
        return lpcShiftPoint, lpcShiftEigVec

    def __checkConvergence(self, thisPathLength, totalPathLength):
        isConverged = False
        R = thisPathLength / totalPathLength
        if R < self.corvergenceRate:
            isConverged = True
        return isConverged


    def solve(self):
        #find start point as points centroid (amplitude-weighted )
        self.isDirectionForwards = True 
        #self.startPoint = np.average(self.pointsXYZ, axis = 0, weights = self.pointsAmpl)
        self.startPoint = np.max(self.pointsXYZ)
        previousPoint = self.__meanShift(self.startPoint)
        self.lpcPoints.append(previousPoint)     
        previousEigVec = None
        nPoints = 1
        totalPathLength = 0
        #forward direction
        while (nPoints < self.maxPoints/2):
            nextPoint, nextEigVec = self.__lpcShift(previousPoint, previousEigVec)
            nextPoint = self.__meanShift(nextPoint)
            thisPathLength = la.norm(nextPoint - previousPoint)
            totalPathLength += thisPathLength
            self.lpcPoints.append(nextPoint)
            previousPoint = nextPoint
            previousEigVec = nextEigVec
            nPoints += 1
            if (self.__checkConvergence(thisPathLength, totalPathLength)):
                break
            
        # back first lpc point (local mean in start point), now go backwards
        self.isDirectionForwards = False
        previousEigVec = None     
        previousPoint = self.lpcPoints[0]
        totalPathLength = 0
        while (nPoints < self.maxPoints):
            nextPoint, nextEigVec = self.__lpcShift(previousPoint, previousEigVec)
            nextPoint = self.__meanShift(nextPoint)
            thisPathLength = la.norm(nextPoint - previousPoint)
            totalPathLength += thisPathLength
            self.lpcPoints.append(nextPoint)
            previousPoint = nextPoint
            previousEigVec = nextEigVec
            nPoints += 1 
            if (self.__checkConvergence(thisPathLength, totalPathLength)):
                break
            
        self.lpcPoints = np.array(self.lpcPoints)
        print(self.lpcPoints.shape)

    def plot(self, truth=None):
        fig = plt.figure()
        ax = plt.axes(projection='3d')
        #ax.scatter3D(self.lpcPoints[:, 0], self.lpcPoints[:,1], self.lpcPoints[:,2], color = 'red')
        ax.scatter3D(self.pointsXYZ[:, 0], self.pointsXYZ[:,1], self.pointsXYZ[:,2], c = self.pointsAmpl, cmap = 'viridis', alpha = 0.3)
        ax.scatter3D(self.lpcPoints[0, 0], self.lpcPoints[0,1], self.lpcPoints[0,2], color = 'green')
        if truth is not None:
            dirh = truth[0]
            dirp = truth[1]
            ax.quiver(*dirh, *dirp*10, color='xkcd:salmon', arrow_length_ratio=0.1, linewidth=2)
        plt.xlabel("x")
        plt.ylabel("y")
        plt.axis('equal')
        plt.show()

In [29]:
lpc = LPCSolver()
data = [[1,2,3],[4,5,6],[7,8,9]]
amps = [10,20,30]
#lpc.setPoints(data, amps)

# data = np.asarray(data)
# amps = np.asarray(amps)

diction = dict(zip(amps,data))

# tuples = [(key, value) for i, (key, value) in enumerate(zip(data, amps))]

# res = dict(tuples)

# print(np.max(list(diction.keys())))

maxAmpPoint = 0
for amp in diction.keys():
    if amp == np.max(list(diction.keys())):
        maxAmpPoint = diction[amp]

print(maxAmpPoint)

[7, 8, 9]
